In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# options
dataset = 'mnist' # options: 'mnist' | 'cifar10'
batch_size = 64   # input batch size for training
epochs = 10       # number of epochs to train
lr = 0.01        # learning rate

In [3]:
# Data Loading
# Warning: this cell might take some time when you run it for the first time, 
#          because it will download the datasets from the internet
if dataset == 'mnist':
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    trainset = datasets.MNIST(root='.', train=True, download=True, transform=data_transform)
    testset = datasets.MNIST(root='.', train=False, download=True, transform=data_transform)

elif dataset == 'cifar10':
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    trainset = datasets.CIFAR10(root='.', train=True, download=True, transform=data_transform)
    testset = datasets.CIFAR10(root='.', train=False, download=True, transform=data_transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
## network and optimizer
if dataset == 'mnist':
    num_inputs = 784
elif dataset == 'cifar10':
    num_inputs = 3072

num_outputs = 10 # same for both CIFAR10 and MNIST, both have 10 classes as outputs

class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(Net, self).__init__()
        self.linear_1 = nn.Linear(num_inputs, 1000)
        self.linear_2 = nn.Linear(1000, num_outputs)
        self.acti = nn.Tanh()

    def forward(self, input):
        input = input.view(-1, num_inputs) # reshape input to batch x num_inputs
        output = self.linear_1(input)
        output = self.linear_2(output)
        output = self.acti(output)
        return output

network = Net(num_inputs, num_outputs)
optimizer = optim.SGD(network.parameters(), lr=lr)

In [5]:
def train(num_epoch):
    network.train()
    for epoch in range(num_epoch):
      print("Epoch:", epoch+1)
      for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    

def test():
    network.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        #data, target = Variable(data, volatile=True), Variable(target)
        output = network(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
        #test_loss += F.cross_entropy(output, target, sum=True).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [6]:
train(10)

Epoch: 1
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.332141
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.237420
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.240619
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.073310
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.048317
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.054746
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.156419
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.135996
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.977255
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.017847
Epoch: 2
Train Epoch: 2 [0/60000 (0%)]	Loss: 1.096754
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.996063
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.133420
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.079706
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.076452
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.032532
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.100202
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.096044
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.040287
Train Epoch: 2 [57600/60000 (96%)]	Loss: 1.0

In [8]:
test()


Test set: Average loss: 0.9764, Accuracy: 9144/10000 (91%)



In [11]:
lr=10
train(10)
network = Net(num_inputs, num_outputs)
optimizer = optim.SGD(network.parameters(), lr=lr)

Epoch: 1
Train Epoch: 1 [0/60000 (0%)]	Loss: 1.017651
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.974151
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.963674
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.906053
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.952112
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.942397
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.021509
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.061133
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.986389
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.024461
Epoch: 2
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.993569
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.915617
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.895260
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.948913
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.967833
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.990103
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.988935
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.011514
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.996715
Train Epoch: 2 [57600/60000 (96%)]	Loss: 1.0

In [12]:
test()


Test set: Average loss: 2.3403, Accuracy: 932/10000 (9%)



# Explanation

We can see the accuracy when lr=10 is considerably low.
This is partly because the learning rate is too big so that it cannot converge to the minimum succussfully.